In [1]:
pip install gym numpy stable-baselines3

Note: you may need to restart the kernel to use updated packages.


In [1]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import time
import os

# Custom 2D racing environment
class RacingEnv(gym.Env):
    def __init__(self):
        super(RacingEnv, self).__init__()
        self.width = 10
        self.height = 20
        self.car_pos = [self.width // 2, 0]
        self.obstacle_pos = [np.random.randint(0, self.width), self.height - 1]
        self.action_space = gym.spaces.Discrete(3)  # Left, Stay, Right
        self.observation_space = gym.spaces.Box(low=0, high=max(self.width, self.height),
                                                shape=(4,), dtype=np.float32)

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.car_pos = [self.width // 2, 0]
        self.obstacle_pos = [np.random.randint(0, self.width), self.height - 1]
        return self._get_obs(), {}

    def step(self, action):
        # Move car
        if action == 0 and self.car_pos[0] > 0:
            self.car_pos[0] -= 1
        elif action == 2 and self.car_pos[0] < self.width - 1:
            self.car_pos[0] += 1

        # Move obstacle
        self.obstacle_pos[1] -= 1

        # Check collision
        if self.car_pos == self.obstacle_pos:
            reward = -10
            terminated = True
        elif self.obstacle_pos[1] < 0:
            reward = 1
            self.obstacle_pos = [np.random.randint(0, self.width), self.height - 1]
            terminated = False
        else:
            reward = 0
            terminated = False

        return self._get_obs(), reward, terminated, False, {}

    def _get_obs(self):
        return np.array([self.car_pos[0], self.car_pos[1],
                         self.obstacle_pos[0], self.obstacle_pos[1]], dtype=np.float32)

    def render(self):
        os.system('cls' if os.name == 'nt' else 'clear')
        for y in range(self.height - 1, -1, -1):
            for x in range(self.width):
                if [x, y] == self.car_pos:
                    print('C', end='')
                elif [x, y] == self.obstacle_pos:
                    print('O', end='')
                else:
                    print('.', end='')
            print()
        print("\n")

# Wrapper to make the environment compatible with stable-baselines3
class VisualRacingEnv(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.env = env

    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        self.env.render()
        return obs, info

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.env.render()
        time.sleep(0.1)  # Add a small delay to make the visualization visible
        return obs, reward, terminated, truncated, info

# Create and wrap the environment
env = DummyVecEnv([lambda: VisualRacingEnv(RacingEnv())])

# Initialize the PPO model
model = PPO("MlpPolicy", env, verbose=1)

# Train the model
model.learn(total_timesteps=100000)

# Evaluate the trained model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

# Save the trained model
model.save("ppo_racing_model")

# Test the model
obs, _ = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, _ = env.reset()

Using cuda device


d:\mini_conda\envs\collabkart\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


....O.....
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
.....C....


..........
....O.....
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
....C.....


..........
..........
....O.....
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
....C.....


..........
..........
..........
....O.....
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
...C......


..........
..........
..........
..........
....O.....
..........
..........
..........
..........
..........
..

In [3]:
pip install 'shimmy>=0.2.1

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'shimmy"


In [1]:
import numpy as np
import torch
from tmrl import get_environment
def analyze_observation(obs):
    print("Analyzing observation structure:")
    
    if isinstance(obs, (list, tuple)):
        print(f"Observation is a {type(obs).__name__} with {len(obs)} elements.")
        for i, item in enumerate(obs):
            print(f"\nElement {i}:")
            analyze_item(item)
    elif isinstance(obs, dict):
        print(f"Observation is a dictionary with {len(obs)} keys.")
        for key, value in obs.items():
            print(f"\nKey: {key}")
            analyze_item(value)
    else:
        analyze_item(obs)

def analyze_item(item):
    if isinstance(item, np.ndarray):
        print(f"Type: NumPy array, Shape: {item.shape}, Data type: {item.dtype}")
    elif isinstance(item, torch.Tensor):
        print(f"Type: PyTorch Tensor, Shape: {item.shape}, Data type: {item.dtype}, Device: {item.device}")
    elif isinstance(item, (int, float, bool)):
        print(f"Type: {type(item).__name__}, Value: {item}")
    elif isinstance(item, str):
        print(f"Type: string, Length: {len(item)}")
    elif isinstance(item, (list, tuple)):
        print(f"Type: {type(item).__name__}, Length: {len(item)}")
        if len(item) > 0:
            print("First element:")
            analyze_item(item[0])
    elif isinstance(item, dict):
        print(f"Type: dictionary, Number of keys: {len(item)}")
        if len(item) > 0:
            first_key = next(iter(item))
            print(f"Example - Key: {first_key}")
            analyze_item(item[first_key])
    else:
        print(f"Type: {type(item).__name__}")
        try:
            print(f"Shape/Length: {len(item)}")
        except:
            print("Shape/Length: Not applicable")

# Example usage
def example_usage(env):
    obs, _ = env.reset()
    analyze_observation(obs)

# Uncomment and use this if you want to test with a specific environment
# from your_environment_module import get_environment
env = get_environment()
example_usage(env)

Analyzing observation structure:
Observation is a tuple with 6 elements.

Element 0:
Type: NumPy array, Shape: (1,), Data type: float32

Element 1:
Type: NumPy array, Shape: (1,), Data type: float32

Element 2:
Type: NumPy array, Shape: (1,), Data type: float32

Element 3:
Type: NumPy array, Shape: (4, 64, 64), Data type: uint8

Element 4:
Type: NumPy array, Shape: (3,), Data type: float32

Element 5:
Type: NumPy array, Shape: (3,), Data type: float32


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random
from tmrl import get_environment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DQN(nn.Module):
    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        conv_out_size = self._get_conv_out(h, w)
        self.fc1 = nn.Linear(conv_out_size + 9, 512)  # 9 additional inputs (1 each from elements 0, 1, 2, and 3 each from elements 4 and 5)
        self.fc2 = nn.Linear(512, outputs)

    def _get_conv_out(self, h, w):
        o = self.conv1(torch.zeros(1, 4, h, w))
        o = self.conv2(o)
        o = self.conv3(o)
        return int(np.prod(o.size()))

    def forward(self, x, additional_inputs):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = torch.cat((x, additional_inputs), dim=1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class DQNAgent:
    def __init__(self, n_actions, memory_size=10000, batch_size=32, gamma=0.99, epsilon_start=1.0, epsilon_final=0.01, epsilon_decay=0.995):
        self.n_actions = n_actions
        self.memory = deque(maxlen=memory_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_final = epsilon_final
        self.epsilon_decay = epsilon_decay

        self.policy_net = DQN(64, 64, n_actions).to(device)
        self.target_net = DQN(64, 64, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters())

    def select_action(self, state):
        if random.random() > self.epsilon:
            with torch.no_grad():
                image = torch.FloatTensor(state[3]).unsqueeze(0).to(device)
                additional = torch.FloatTensor(np.concatenate([state[0], state[1], state[2], state[4], state[5]])).unsqueeze(0).to(device)
                return self.policy_net(image, additional).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = zip(*batch)

        state_image_batch = torch.FloatTensor(np.stack([s[3] for s in state_batch])).to(device)
        state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in state_batch])).to(device)

        next_state_image_batch = torch.FloatTensor(np.stack([s[3] for s in next_state_batch])).to(device)
        next_state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in next_state_batch])).to(device)

        action_batch = torch.LongTensor(action_batch).to(device)
        reward_batch = torch.FloatTensor(reward_batch).to(device)
        done_batch = torch.FloatTensor(done_batch).to(device)

        q_values = self.policy_net(state_image_batch, state_additional_batch).gather(1, action_batch.unsqueeze(1))
        next_q_values = self.target_net(next_state_image_batch, next_state_additional_batch).max(1)[0].detach()
        expected_q_values = reward_batch + (1 - done_batch) * self.gamma * next_q_values

        loss = F.smooth_l1_loss(q_values, expected_q_values.unsqueeze(1))

        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        self.epsilon = max(self.epsilon_final, self.epsilon * self.epsilon_decay)

    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

def preprocess_observation(obs):
    # Ensure all elements are numpy arrays
    speed = np.array(obs[0]).flatten()
    steering = np.array(obs[1]).flatten()
    gyro = np.array(obs[2]).flatten()
    
    # Normalize image
    image = np.array(obs[3]).astype(np.float32) / 255.0
    
    # Ensure previous actions and current actions are numpy arrays
    prev_action = np.array(obs[4]).flatten()
    action = np.array(obs[5]).flatten()
    
    # Normalize to [-1, 1] range
    speed = np.clip(speed / 300.0, -1, 1)  # Assuming max speed is 300
    steering = np.clip(steering / np.pi, -1, 1)  # Assuming steering is in radians
    gyro = np.clip(gyro / np.pi, -1, 1)  # Assuming gyro is in radians
    prev_action = np.clip(prev_action, -1, 1)
    action = np.clip(action, -1, 1)
    
    return (speed, steering, gyro, image, prev_action, action)

def env_action_to_agent_action(env_action, n_actions):
    # Convert environment action to agent action index
    if isinstance(env_action, (int, np.integer)):
        return env_action
    elif isinstance(env_action, (float, np.float32, np.float64)):
        # Discretize the continuous action
        return int((env_action + 1) * (n_actions - 1) / 2)
    elif isinstance(env_action, np.ndarray):
        return np.argmax(env_action)
    else:
        raise ValueError(f"Unexpected action type: {type(env_action)}")

def agent_action_to_env_action(agent_action, n_actions):
    # Convert agent action index to environment action
    steering = (agent_action * 2 / (n_actions - 1)) - 1
    throttle = 1.0  # Full throttle
    brake = 0.0     # No brake
    return [ throttle, brake,steering]  # Return a list with three values

def train(env, agent, num_episodes, max_steps_per_episode):
    for episode in range(num_episodes):
        obs, _ = env.reset()
        state = preprocess_observation(obs)
        total_reward = 0

        for step in range(max_steps_per_episode):
            agent_action = agent.select_action(state)
            env_action = agent_action_to_env_action(agent_action.item(), agent.n_actions)
            next_obs, reward, terminated, truncated, _ = env.step(env_action)  # Pass env_action as a list
            next_state = preprocess_observation(next_obs)
            done = terminated or truncated
            
            agent.remember(state, agent_action.item(), reward, next_state, done)
            agent.replay()
            
            state = next_state
            total_reward += reward

            if done:
                break

        if episode % 10 == 0:
            agent.update_target_network()

        print(f"Episode {episode}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.2f}")# Usage example:
# from your_environment import get_environment
env = get_environment()
n_actions = 3  # Number of possible actions (adjust as needed)
agent = DQNAgent(n_actions)
train(env, agent, num_episodes=1000, max_steps_per_episode=1000)

d:\mini_conda\envs\collabkart\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6700939998991089
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Episode 0, Total Reward: 3.4399999901652336, Epsilon: 0.78
Episode 1, Total Reward: 11.859999973326921, Epsilon: 0.36
Episode 2, Total Reward: 138.37999992072582, Epsilon: 0.04
Episode 3, Total Reward: 1.88999998383224, Epsilon: 0.03
Episode 4, Total Reward: 10.640000021085143, Epsilon: 0.01
Episode 5, Total Reward: 1.4599999822676182, Epsilon: 0.01
Episode 6, Total Reward: 10.85999995842576, Epsilon: 0.01
Episode 7, Total Reward: 2.219999987632036, Epsilon: 0.01
Episode 8, Total Reward: 1.3199999928474426, Epsilon: 0.01
Episode 9, Total Reward: 12.089999966323376, Epsilon: 0.01
Episode 10, Total Reward: 1.059999991208315, Epsilon: 0.01
Episode 11, Total Reward: 1.149999987334013, Epsilon: 0.01
Episode 12, Total Reward: 1.1799999866634607, Epsilon: 0.01
Episode 13, Total Reward: 1.6499999817460775, Epsilon: 0.01
Episode 14, Total Reward: 1.0899999868124723, Epsilon: 0.01
Episode 15, Total Reward: 1.0499999951571226, Epsilon: 0.01
Episode 16, Total Reward: 1.0499999951571226, Epsilon: 0

d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.051254299040010665
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Episode 81, Total Reward: 138.37999986857176, Epsilon: 0.01
Episode 82, Total Reward: 3.9499999713152647, Epsilon: 0.01
Episode 83, Total Reward: 1.1699999943375587, Epsilon: 0.01
Episode 84, Total Reward: 28.000000031664968, Epsilon: 0.01
Episode 85, Total Reward: 1.109999991953373, Epsilon: 0.01
Episode 86, Total Reward: 1.239999983459711, Epsilon: 0.01
Episode 87, Total Reward: 2.179999992251396, Epsilon: 0.01
Episode 88, Total Reward: 11.330000007525086, Epsilon: 0.01
Episode 89, Total Reward: 1.55999999307096, Epsilon: 0.01
Episode 90, Total Reward: 3.859999969601631, Epsilon: 0.01
Episode 91, Total Reward: 5.549999983981252, Epsilon: 0.01
Episode 92, Total Reward: 1.1199999935925007, Epsilon: 0.01
Episode 93, Total Reward: 4.0999999940395355, Epsilon: 0.01
Episode 94, Total Reward: 2.709999991580844, Epsilon: 0.01
Episode 95, Total Reward: 12.259999975562096, Epsilon: 0.01
Episode 96, Total Reward: 1.2099999897181988, Epsilon: 0.01
Episode 97, Total Reward: 138.3800022304058, Eps

d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.053198899629933294
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.09694309999031248
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Episode 146, Total Reward: 31.36999993212521, Epsilon: 0.01
Episode 147, Total Reward: 1.1399999875575304, Epsilon: 0.01
Episode 148, Total Reward: 4.319999974220991, Epsilon: 0.01
Episode 149, Total Reward: 28.03999998793006, Epsilon: 0.01
Episode 150, Total Reward: 4.519999975338578, Epsilon: 0.01
Episode 151, Total Reward: 1.2399999797344208, Epsilon: 0.01
Episode 152, Total Reward: 3.5499999802559614, Epsilon: 0.01
Episode 153, Total Reward: 19.169999981299043, Epsilon: 0.01
Episode 154, Total Reward: 15.419999964535236, Epsilon: 0.01
Episode 155, Total Reward: 3.299999974668026, Epsilon: 0.01
Episode 156, Total Reward: 1.9399999808520079, Epsilon: 0.01
Episode 157, Total Reward: 1.1599999852478504, Epsilon: 0.01
Episode 158, Total Reward: 1.1699999943375587, Epsilon: 0.01
Episode 159, Total Reward: 1.2199999932199717, Epsilon: 0.01
Episode 160, Total Reward: 10.690000031143427, Epsilon: 0.01


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random
from tmrl import get_environment
import os
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.ln1 = nn.GroupNorm(32, channels)  # Using GroupNorm instead of LayerNorm
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.ln2 = nn.GroupNorm(32, channels)  # Using GroupNorm instead of LayerNorm

    def forward(self, x):
        residual = x
        out = F.relu(self.ln1(self.conv1(x)))
        out = self.ln2(self.conv2(out))
        out += residual
        return F.relu(out)

class AttentionModule(nn.Module):
    def __init__(self, in_channels):
        super(AttentionModule, self).__init__()
        self.conv = nn.Conv2d(in_channels, 1, kernel_size=1)
        self.softmax = nn.Softmax(dim=-1)  # Use dim=-1 for the final dimension

    def forward(self, x):
        batch_size, C, H, W = x.size()
        proj = self.conv(x).view(batch_size, 1, -1)
        weights = self.softmax(proj).view(batch_size, 1, H, W)
        return x * weights.expand_as(x)


class ComplexDQN(nn.Module):
    def __init__(self, h, w, outputs):
        super(ComplexDQN, self).__init__()
        
        # Initial convolutional layers
        self.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2)
        self.bn2 = nn.BatchNorm2d(128)
        
        # Residual blocks
        self.res1 = ResidualBlock(128)
        self.res2 = ResidualBlock(128)
        self.res3 = ResidualBlock(128)
        
        # Attention module
        self.attention = AttentionModule(128)
        
        # Additional convolutional layers
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(512)

        # Calculate the size of flattened features
        self.conv_out_size = self._get_conv_out(h, w)

        # Fully connected layers
        self.fc1 = nn.Linear(self.conv_out_size + 9, 2048)
        self.ln1 = nn.LayerNorm(2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.ln2 = nn.LayerNorm(1024)
        self.fc3 = nn.Linear(1024, 512)
        self.ln3 = nn.LayerNorm(512)
        self.fc4 = nn.Linear(512, outputs)

        # Dropout layers
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.4)
        self.dropout3 = nn.Dropout(0.4)

    def _get_conv_out(self, h, w):
        o = F.relu(self.bn1(self.conv1(torch.zeros(1, 4, h, w))))
        o = F.relu(self.bn2(self.conv2(o)))
        o = self.res1(o)
        o = self.res2(o)
        o = self.res3(o)
        o = self.attention(o)
        o = F.relu(self.bn3(self.conv3(o)))
        o = F.relu(self.bn4(self.conv4(o)))
        return int(np.prod(o.size()))

    def forward(self, x, additional_inputs):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)
        x = self.attention(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = torch.cat((x, additional_inputs), dim=1)
        x = F.relu(self.ln1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.ln2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.relu(self.ln3(self.fc3(x)))
        x = self.dropout3(x)
        return self.fc4(x)


class ComplexDQNAgent:
    def __init__(self, n_actions, memory_size=200000, batch_size=128, gamma=0.99, epsilon_start=1.0, epsilon_final=0.01, epsilon_decay=0.9995, learning_rate=0.0001):
        self.n_actions = n_actions
        self.memory = deque(maxlen=memory_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_final = epsilon_final
        self.epsilon_decay = epsilon_decay
        self.steps = 0

        self.policy_net = ComplexDQN(64, 64, n_actions).to(device)
        self.target_net = ComplexDQN(64, 64, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=200000, eta_min=1e-6)

    def select_action(self, state):
        if random.random() > self.epsilon:
            with torch.no_grad():
                image = torch.FloatTensor(state[3]).unsqueeze(0).to(device)
                additional = torch.FloatTensor(np.concatenate([state[0], state[1], state[2], state[4], state[5]])).unsqueeze(0).to(device)
                return self.policy_net(image, additional).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = zip(*batch)

        state_image_batch = torch.FloatTensor(np.stack([s[3] for s in state_batch])).to(device)
        state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in state_batch])).to(device)

        next_state_image_batch = torch.FloatTensor(np.stack([s[3] for s in next_state_batch])).to(device)
        next_state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in next_state_batch])).to(device)

        action_batch = torch.LongTensor(action_batch).to(device)
        reward_batch = torch.FloatTensor(reward_batch).to(device)
        done_batch = torch.FloatTensor(done_batch).to(device)

        q_values = self.policy_net(state_image_batch, state_additional_batch).gather(1, action_batch.unsqueeze(1))
        next_q_values = self.target_net(next_state_image_batch, next_state_additional_batch).max(1)[0].detach()
        expected_q_values = reward_batch + (1 - done_batch) * self.gamma * next_q_values

        loss = F.smooth_l1_loss(q_values, expected_q_values.unsqueeze(1))

        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        self.epsilon = max(self.epsilon_final, self.epsilon * self.epsilon_decay)

    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    def save_checkpoint(self, episode, directory="checkpoints"):
        if not os.path.exists(directory):
            os.makedirs(directory)
        checkpoint = {
            'episode': episode,
            'model_state_dict': self.policy_net.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'epsilon': self.epsilon,
            'steps': self.steps
        }
        filename = os.path.join(directory, f"checkpoint_episode_{episode}_step_{self.steps}.pth")
        torch.save(checkpoint, filename)
        print(f"Checkpoint saved: {filename}")

    def load_checkpoint(self, filename):
        checkpoint = torch.load(filename)
        self.policy_net.load_state_dict(checkpoint['model_state_dict'])
        self.target_net.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.epsilon = 0.9
        self.steps = checkpoint['steps']
        return checkpoint['episode']

def preprocess_observation(obs):
    speed = np.array(obs[0]).flatten()
    steering = np.array(obs[1]).flatten()
    gyro = np.array(obs[2]).flatten()
    image = np.array(obs[3]).astype(np.float32) / 255.0
    prev_action = np.array(obs[4]).flatten()
    action = np.array(obs[5]).flatten()
    
    speed = np.clip(speed / 300.0, -1, 1)
    steering = np.clip(steering / np.pi, -1, 1)
    gyro = np.clip(gyro / np.pi, -1, 1)
    prev_action = np.clip(prev_action, -1, 1)
    action = np.clip(action, -1, 1)
    
    return (speed, steering, gyro, image, prev_action, action)

def env_action_to_agent_action(env_action, n_actions):
    if isinstance(env_action, (int, np.integer)):
        return env_action
    elif isinstance(env_action, (float, np.float32, np.float64)):
        return int((env_action + 1) * (n_actions - 1) / 2)
    elif isinstance(env_action, np.ndarray):
        return np.argmax(env_action)
    else:
        raise ValueError(f"Unexpected action type: {type(env_action)}")

def agent_action_to_env_action(agent_action, n_actions):
    steering = (agent_action * 2 / (n_actions - 1)) - 1
    throttle = 1.0
    brake = 0.0
    return [throttle, brake, steering]


def detect_crash(obs, prev_obs, speed_threshold=1.0):
    """
    Detect if a crash has occurred based on observations.
    
    :param obs: Current observation
    :param prev_obs: Previous observation
    :param speed_threshold: Threshold for sudden speed change
    :return: True if crash detected, False otherwise
    """
    if prev_obs is None:
        return False
    
    current_speed = np.linalg.norm(obs[0])  # Assuming obs[0] contains speed information
    prev_speed = np.linalg.norm(prev_obs[0])
    
    # Detect sudden drop in speed
    if prev_speed - current_speed > speed_threshold:
        return True
    

def train(env, agent, num_episodes, max_steps_per_episode):
    for episode in range(num_episodes):
        obs, _ = env.reset()
        state = preprocess_observation(obs)
        total_reward = 0
        episode_start_time = time.time()
        prev_obs = None
        crashes = 0

        for step in range(max_steps_per_episode):
            agent_action = agent.select_action(state)
            env_action = agent_action_to_env_action(agent_action.item(), agent.n_actions)
            next_obs, env_reward, terminated, truncated, _ = env.step(env_action)
            next_state = preprocess_observation(next_obs)
            
            # Detect crash
            if detect_crash(next_obs, prev_obs):
                crashes += 1
                env_reward -= 10  # Apply a penalty for crashing
            
            done = terminated or truncated
            
            # Calculate time-based reward
            elapsed_time = time.time() - episode_start_time
            time_factor = 1 / (elapsed_time + 1)  # Adding 1 to avoid division by zero
            reward = env_reward * time_factor * 10  # Multiplying by 10 to make time factor more significant
            
            agent.remember(state, agent_action.item(), reward, next_state, done)
            agent.replay()
            
            state = next_state
            prev_obs = next_obs
            total_reward += reward

            agent.steps += 1
            agent.scheduler.step()  # Update learning rate

            # Save checkpoint every 1000 steps
            if agent.steps % 1000 == 0:
                agent.save_checkpoint(episode)

            if done:
                break

        if episode % 5 == 0:
            agent.update_target_network()

        print(f"Episode {episode}, Total Reward: {total_reward:.2f}, Crashes: {crashes}, Epsilon: {agent.epsilon:.4f}, LR: {agent.scheduler.get_last_lr()[0]:.6f}, Time: {elapsed_time:.2f}s")

if __name__ == "__main__":
    env = get_environment()
    n_actions = 3  # Number of possible actions (adjust as needed)
    agent = ComplexDQNAgent(n_actions)
    agent.load_checkpoint("checkpoints\checkpoint_episode_285_step_38000.pth")
    train(env, agent, num_episodes=5000, max_steps_per_episode=5000)

cuda


d:\mini_conda\envs\collabkart\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.43608859993400984
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Episode 0, Total Reward: -102.13, Crashes: 3, Epsilon: 0.9000, LR: 0.000091, Time: 4.74s


d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.8637980998319108
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.5705845000047702
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.5111443999921903
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.5738961000024574
  warnings.warn(f"Time-step timed out. Elapsed since last time-step:

Episode 1, Total Reward: -11.43, Crashes: 2, Epsilon: 0.8924, LR: 0.000091, Time: 13.72s


d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6091880000021774
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6108299999905284
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6119638000091072
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.5941396999842254
  warnings.warn(f"Time-step timed out. Elapsed since last time-step:

Episode 2, Total Reward: -96.86, Crashes: 17, Epsilon: 0.8608, LR: 0.000091, Time: 45.33s


d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6207224999961909
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6092500999948243
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.61295739999332
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.5915233999985503
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {

Episode 3, Total Reward: -33.63, Crashes: 2, Epsilon: 0.8304, LR: 0.000091, Time: 45.72s


d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6020728999865241
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6060378999973182
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6000230999925407
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.5874723000015365
  warnings.warn(f"Time-step timed out. Elapsed since last time-step:

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random
from tmrl import get_environment
import os
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.ln1 = nn.GroupNorm(32, channels)  # Using GroupNorm instead of LayerNorm
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.ln2 = nn.GroupNorm(32, channels)  # Using GroupNorm instead of LayerNorm

    def forward(self, x):
        residual = x
        out = F.relu(self.ln1(self.conv1(x)))
        out = self.ln2(self.conv2(out))
        out += residual
        return F.relu(out)

class AttentionModule(nn.Module):
    def __init__(self, in_channels):
        super(AttentionModule, self).__init__()
        self.conv = nn.Conv2d(in_channels, 1, kernel_size=1)
        self.softmax = nn.Softmax(dim=-1)  # Use dim=-1 for the final dimension

    def forward(self, x):
        batch_size, C, H, W = x.size()
        proj = self.conv(x).view(batch_size, 1, -1)
        weights = self.softmax(proj).view(batch_size, 1, H, W)
        return x * weights.expand_as(x)


class ComplexDQN(nn.Module):
    def __init__(self, h, w, outputs):
        super(ComplexDQN, self).__init__()
        
        # Initial convolutional layers
        self.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2)
        self.bn2 = nn.BatchNorm2d(128)
        
        # Residual blocks
        self.res1 = ResidualBlock(128)
        self.res2 = ResidualBlock(128)
        self.res3 = ResidualBlock(128)
        
        # Attention module
        self.attention = AttentionModule(128)
        
        # Additional convolutional layers
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(512)

        # Calculate the size of flattened features
        self.conv_out_size = self._get_conv_out(h, w)

        # Fully connected layers
        self.fc1 = nn.Linear(self.conv_out_size + 9, 2048)
        self.ln1 = nn.LayerNorm(2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.ln2 = nn.LayerNorm(1024)
        self.fc3 = nn.Linear(1024, 512)
        self.ln3 = nn.LayerNorm(512)
        self.fc4 = nn.Linear(512, outputs)

        # Dropout layers
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.4)
        self.dropout3 = nn.Dropout(0.4)

    def _get_conv_out(self, h, w):
        o = F.relu(self.bn1(self.conv1(torch.zeros(1, 4, h, w))))
        o = F.relu(self.bn2(self.conv2(o)))
        o = self.res1(o)
        o = self.res2(o)
        o = self.res3(o)
        o = self.attention(o)
        o = F.relu(self.bn3(self.conv3(o)))
        o = F.relu(self.bn4(self.conv4(o)))
        return int(np.prod(o.size()))

    def forward(self, x, additional_inputs):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)
        x = self.attention(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = torch.cat((x, additional_inputs), dim=1)
        x = F.relu(self.ln1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.ln2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.relu(self.ln3(self.fc3(x)))
        x = self.dropout3(x)
        return self.fc4(x)


class ComplexDQNAgent:
    def __init__(self, n_actions, memory_size=200000, batch_size=128, gamma=0.99, epsilon_start=1.0, epsilon_final=0.01, epsilon_decay=0.9995, learning_rate=0.0001):
        self.n_actions = n_actions
        self.memory = deque(maxlen=memory_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_final = epsilon_final
        self.epsilon_decay = epsilon_decay
        self.steps = 0

        self.policy_net = ComplexDQN(64, 64, n_actions).to(device)
        self.target_net = ComplexDQN(64, 64, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=200000, eta_min=1e-6)

    def select_action(self, state):
        if random.random() > self.epsilon:
            with torch.no_grad():
                image = torch.FloatTensor(state[3]).unsqueeze(0).to(device)
                additional = torch.FloatTensor(np.concatenate([state[0], state[1], state[2], state[4], state[5]])).unsqueeze(0).to(device)
                return self.policy_net(image, additional).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = zip(*batch)

        state_image_batch = torch.FloatTensor(np.stack([s[3] for s in state_batch])).to(device)
        state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in state_batch])).to(device)

        next_state_image_batch = torch.FloatTensor(np.stack([s[3] for s in next_state_batch])).to(device)
        next_state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in next_state_batch])).to(device)

        action_batch = torch.LongTensor(action_batch).to(device)
        reward_batch = torch.FloatTensor(reward_batch).to(device)
        done_batch = torch.FloatTensor(done_batch).to(device)

        q_values = self.policy_net(state_image_batch, state_additional_batch).gather(1, action_batch.unsqueeze(1))
        next_q_values = self.target_net(next_state_image_batch, next_state_additional_batch).max(1)[0].detach()
        expected_q_values = reward_batch + (1 - done_batch) * self.gamma * next_q_values

        loss = F.smooth_l1_loss(q_values, expected_q_values.unsqueeze(1))

        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        self.epsilon = max(self.epsilon_final, self.epsilon * self.epsilon_decay)

    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    def save_checkpoint(self, episode, directory="checkpoints"):
        if not os.path.exists(directory):
            os.makedirs(directory)
        checkpoint = {
            'episode': episode,
            'model_state_dict': self.policy_net.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'epsilon': self.epsilon,
            'steps': self.steps
        }
        filename = os.path.join(directory, f"checkpoint_episode_{episode}_step_{self.steps}.pth")
        torch.save(checkpoint, filename)
        print(f"Checkpoint saved: {filename}")

    def load_checkpoint(self, filename):
        checkpoint = torch.load(filename)
        self.policy_net.load_state_dict(checkpoint['model_state_dict'])
        self.target_net.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.epsilon = 0.9
        self.steps = checkpoint['steps']
        return checkpoint['episode']


cuda


In [ ]:

class SimpleDQN(nn.Module):
    def __init__(self, h, w, outputs):
        super(SimpleDQN, self).__init__()
        
        # Simplified convolutional layers
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        
        # Calculate the size of flattened features
        conv_out_size = self._get_conv_out(h, w)
        
        # Simplified fully connected layers
        self.fc1 = nn.Linear(conv_out_size + 9, 512)
        self.fc2 = nn.Linear(512, outputs)

    def _get_conv_out(self, h, w):
        o = F.relu(self.conv1(torch.zeros(1, 4, h, w)))
        o = F.relu(self.conv2(o))
        o = F.relu(self.conv3(o))
        return int(np.prod(o.size()))

    def forward(self, x, additional_inputs):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = torch.cat((x, additional_inputs), dim=1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class SimpleDQNAgent:
    def __init__(self, n_actions, memory_size=100000, batch_size=32, gamma=0.99, epsilon_start=1.0, epsilon_final=0.01, epsilon_decay=0.995, learning_rate=0.001):
        self.n_actions = n_actions
        self.memory = deque(maxlen=memory_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_final = epsilon_final
        self.epsilon_decay = epsilon_decay
        self.steps = 0

        self.policy_net = SimpleDQN(64, 64, n_actions).to(device)
        self.target_net = SimpleDQN(64, 64, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)

    def select_action(self, state):
        if random.random() > self.epsilon:
            with torch.no_grad():
                image = torch.FloatTensor(state[3]).unsqueeze(0).to(device)
                additional = torch.FloatTensor(np.concatenate([state[0], state[1], state[2], state[4], state[5]])).unsqueeze(0).to(device)
                return self.policy_net(image, additional).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = zip(*batch)

        state_image_batch = torch.FloatTensor(np.stack([s[3] for s in state_batch])).to(device)
        state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in state_batch])).to(device)

        next_state_image_batch = torch.FloatTensor(np.stack([s[3] for s in next_state_batch])).to(device)
        next_state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in next_state_batch])).to(device)

        action_batch = torch.LongTensor(action_batch).to(device)
        reward_batch = torch.FloatTensor(reward_batch).to(device)
        done_batch = torch.FloatTensor(done_batch).to(device)

        q_values = self.policy_net(state_image_batch, state_additional_batch).gather(1, action_batch.unsqueeze(1))
        next_q_values = self.target_net(next_state_image_batch, next_state_additional_batch).max(1)[0].detach()
        expected_q_values = reward_batch + (1 - done_batch) * self.gamma * next_q_values

        loss = F.smooth_l1_loss(q_values, expected_q_values.unsqueeze(1))

        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        self.epsilon = max(self.epsilon_final, self.epsilon * self.epsilon_decay)

    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    def save_checkpoint(self, episode, directory="checkpoints_simple"):
        if not os.path.exists(directory):
            os.makedirs(directory)
        checkpoint = {
            'episode': episode,
            'model_state_dict': self.policy_net.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'epsilon': self.epsilon,
            'steps': self.steps
        }
        filename = os.path.join(directory, f"checkpoint_episode_{episode}_step_{self.steps}.pth")
        torch.save(checkpoint, filename)
        print(f"Checkpoint saved: {filename}")

    def load_checkpoint(self, filename):
        checkpoint = torch.load(filename)
        self.policy_net.load_state_dict(checkpoint['model_state_dict'])
        self.target_net.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.epsilon = checkpoint['epsilon']
        self.steps = checkpoint['steps']
        return checkpoint['episode']

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import deque
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ComplexDQN(nn.Module):
    def __init__(self, h, w, outputs, additional_inputs=9):
        super(ComplexDQN, self).__init__()
        
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        conv_out_size = self._get_conv_out(h, w)
        
        self.fc1 = nn.Linear(conv_out_size + additional_inputs, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, outputs)
        
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.3)

    def _get_conv_out(self, h, w):
        o = F.relu(self.bn1(self.conv1(torch.zeros(1, 4, h, w))))
        o = F.relu(self.bn2(self.conv2(o)))
        o = F.relu(self.bn3(self.conv3(o)))
        return int(np.prod(o.size()))

    def forward(self, x, additional_inputs):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        
        x = x.view(x.size(0), -1)
        x = torch.cat((x, additional_inputs), dim=1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        return self.fc3(x)

class ComplexDQNAgent:
    def __init__(self, n_actions, memory_size=100000, batch_size=64, gamma=0.99, 
                 epsilon_start=1.0, epsilon_final=0.01, epsilon_decay=0.999, 
                 learning_rate=0.0005, target_update=10, 
                 checkpoint_dir="checkpoints_complex"):
        self.n_actions = n_actions
        self.memory = deque(maxlen=memory_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_final = epsilon_final
        self.epsilon_decay = epsilon_decay
        self.target_update = target_update
        self.checkpoint_dir = checkpoint_dir
        self.steps = 0

        # Initialize networks
        self.policy_net = ComplexDQN(64, 64, n_actions).to(device)
        self.target_net = ComplexDQN(64, 64, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10000, gamma=0.95)

    def select_action(self, state):
        if random.random() > self.epsilon:
            with torch.no_grad():
                image = torch.FloatTensor(state[3]).unsqueeze(0).to(device)
                additional = torch.FloatTensor(np.concatenate([state[0], state[1], state[2], state[4], state[5]])).unsqueeze(0).to(device)
                return self.policy_net(image, additional).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = zip(*batch)

        state_image_batch = torch.FloatTensor(np.stack([s[3] for s in state_batch])).to(device)
        state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in state_batch])).to(device)

        next_state_image_batch = torch.FloatTensor(np.stack([s[3] for s in next_state_batch])).to(device)
        next_state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in next_state_batch])).to(device)

        action_batch = torch.LongTensor(action_batch).to(device)
        reward_batch = torch.FloatTensor(reward_batch).to(device)
        done_batch = torch.FloatTensor(done_batch).to(device)

        state_action_values = self.policy_net(state_image_batch, state_additional_batch).gather(1, action_batch.unsqueeze(1))
        next_state_values = self.target_net(next_state_image_batch, next_state_additional_batch).max(1)[0].detach()
        expected_state_action_values = reward_batch + (1 - done_batch) * self.gamma * next_state_values

        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), max_norm=1.0)
        self.optimizer.step()
        self.scheduler.step()

        self.epsilon = max(self.epsilon_final, self.epsilon * self.epsilon_decay)

    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    def save_checkpoint(self, episode):
        if not os.path.exists(self.checkpoint_dir):
            os.makedirs(self.checkpoint_dir)
        checkpoint = {
            'episode': episode,
            'model_state_dict': self.policy_net.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'epsilon': self.epsilon,
            'steps': self.steps
        }
        filename = os.path.join(self.checkpoint_dir, f"checkpoint_episode_{episode}_step_{self.steps}.pth")
        torch.save(checkpoint, filename)
        print(f"Checkpoint saved: {filename}")

    def load_checkpoint(self, filename):
        checkpoint = torch.load(filename)
        self.policy_net.load_state_dict(checkpoint['model_state_dict'])
        self.target_net.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.epsilon = checkpoint['epsilon']
        self.steps = checkpoint['steps']
        return checkpoint['episode']

# The train and evaluate methods remain the same as in the previous implementation

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random
from tmrl import get_environment
import os
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


class SimpleDQN(nn.Module):
    def __init__(self, h, w, outputs):
        super(SimpleDQN, self).__init__()
        
        # Simplified convolutional layers
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        
        # Calculate the size of flattened features
        conv_out_size = self._get_conv_out(h, w)
        
        # Simplified fully connected layers
        self.fc1 = nn.Linear(conv_out_size + 9, 512)
        self.fc2 = nn.Linear(512, outputs)

    def _get_conv_out(self, h, w):
        o = F.relu(self.conv1(torch.zeros(1, 4, h, w)))
        o = F.relu(self.conv2(o))
        o = F.relu(self.conv3(o))
        return int(np.prod(o.size()))

    def forward(self, x, additional_inputs):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = torch.cat((x, additional_inputs), dim=1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class SimpleDQNAgent:
    def __init__(self, n_actions, memory_size=100000, batch_size=32, gamma=0.99, epsilon_start=1.0, epsilon_final=0.01, epsilon_decay=0.995, learning_rate=0.001):
        self.n_actions = n_actions
        self.memory = deque(maxlen=memory_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_final = epsilon_final
        self.epsilon_decay = epsilon_decay
        self.steps = 0

        self.policy_net = SimpleDQN(64, 64, n_actions).to(device)
        self.target_net = SimpleDQN(64, 64, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)

    def select_action(self, state):
        if random.random() > self.epsilon:
            with torch.no_grad():
                image = torch.FloatTensor(state[3]).unsqueeze(0).to(device)
                additional = torch.FloatTensor(np.concatenate([state[0], state[1], state[2], state[4], state[5]])).unsqueeze(0).to(device)
                return self.policy_net(image, additional).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = zip(*batch)

        state_image_batch = torch.FloatTensor(np.stack([s[3] for s in state_batch])).to(device)
        state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in state_batch])).to(device)

        next_state_image_batch = torch.FloatTensor(np.stack([s[3] for s in next_state_batch])).to(device)
        next_state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in next_state_batch])).to(device)

        action_batch = torch.LongTensor(action_batch).to(device)
        reward_batch = torch.FloatTensor(reward_batch).to(device)
        done_batch = torch.FloatTensor(done_batch).to(device)

        q_values = self.policy_net(state_image_batch, state_additional_batch).gather(1, action_batch.unsqueeze(1))
        next_q_values = self.target_net(next_state_image_batch, next_state_additional_batch).max(1)[0].detach()
        expected_q_values = reward_batch + (1 - done_batch) * self.gamma * next_q_values

        loss = F.smooth_l1_loss(q_values, expected_q_values.unsqueeze(1))

        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        self.epsilon = max(self.epsilon_final, self.epsilon * self.epsilon_decay)

    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    def save_checkpoint(self, episode, directory="checkpoints_simple_left_turn"):
        if not os.path.exists(directory):
            os.makedirs(directory)
        checkpoint = {
            'episode': episode,
            'model_state_dict': self.policy_net.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'epsilon': self.epsilon,
            'steps': self.steps
        }
        filename = os.path.join(directory, f"checkpoint_episode_{episode}_step_{self.steps}.pth")
        torch.save(checkpoint, filename)
        print(f"Checkpoint saved: {filename}")

    def load_checkpoint(self, filename):
        checkpoint = torch.load(filename)
        self.policy_net.load_state_dict(checkpoint['model_state_dict'])
        self.target_net.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.epsilon = checkpoint['epsilon']
        self.steps = checkpoint['steps']
        return checkpoint['episode']

def preprocess_observation(obs):
    speed = np.array(obs[0]).flatten()
    steering = np.array(obs[1]).flatten()
    gyro = np.array(obs[2]).flatten()
    image = np.array(obs[3]).astype(np.float32) / 255.0
    prev_action = np.array(obs[4]).flatten()
    action = np.array(obs[5]).flatten()
    
    speed = np.clip(speed / 300.0, -1, 1)
    steering = np.clip(steering / np.pi, -1, 1)
    gyro = np.clip(gyro / np.pi, -1, 1)
    prev_action = np.clip(prev_action, -1, 1)
    action = np.clip(action, -1, 1)
    
    return (speed, steering, gyro, image, prev_action, action)

def env_action_to_agent_action(env_action, n_actions):
    if isinstance(env_action, (int, np.integer)):
        return env_action
    elif isinstance(env_action, (float, np.float32, np.float64)):
        return int((env_action + 1) * (n_actions - 1) / 2)
    elif isinstance(env_action, np.ndarray):
        return np.argmax(env_action)
    else:
        raise ValueError(f"Unexpected action type: {type(env_action)}")

def agent_action_to_env_action(agent_action, n_actions):
    steering = (agent_action * 2 / (n_actions - 1)) - 1
    throttle = 1.0
    brake = 0.0
    return [throttle, brake, steering]

def detect_crash(obs, prev_obs, speed_threshold=1.0):
    if prev_obs is None:
        return False
    
    current_speed = np.linalg.norm(obs[0])
    prev_speed = np.linalg.norm(prev_obs[0])
    
    if prev_speed - current_speed > speed_threshold:
        return True
    
    return False

def train(env, agent, num_episodes, max_steps_per_episode):
    for episode in range(num_episodes):
        obs, _ = env.reset()
        state = preprocess_observation(obs)
        total_reward = 0
        episode_start_time = time.time()
        prev_obs = None
        crashes = 0

        for step in range(max_steps_per_episode):
            agent_action = agent.select_action(state)
            env_action = agent_action_to_env_action(agent_action.item(), agent.n_actions)
            next_obs, env_reward, terminated, truncated, _ = env.step(env_action)
            next_state = preprocess_observation(next_obs)
            
            if detect_crash(next_obs, prev_obs):
                crashes += 1
                env_reward-= 10
            done = terminated or truncated
            
            elapsed_time = time.time() - episode_start_time
            time_factor = 1 / (elapsed_time + 1)
            reward = (env_reward)
            
            agent.remember(state, agent_action.item(), reward, next_state, done)
            agent.replay()
            
            state = next_state
            prev_obs = next_obs
            total_reward += reward

            agent.steps += 1

            if agent.steps % 1000 == 0:
                agent.save_checkpoint(episode)

            if done:
                break
        # total_reward-= 20*crashes
        if episode % 10 == 0:
            agent.update_target_network()

        print(f"Episode {episode}, Total Reward: {total_reward:.2f}, Crashes: {crashes}, Epsilon: {agent.epsilon:.4f}, Time: {elapsed_time:.2f}s")

if __name__ == "__main__":
    env = get_environment()
    n_actions = 3
    agent = SimpleDQNAgent(n_actions)
    train(env, agent, num_episodes=5000, max_steps_per_episode=5000)

cuda


d:\mini_conda\envs\collabkart\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6878618984483182
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Episode 0, Total Reward: 472.86, Crashes: 15, Epsilon: 0.2799, Time: 15.64s
Episode 1, Total Reward: 22.60, Crashes: 7, Epsilon: 0.1320, Time: 8.20s
Episode 2, Total Reward: 56.46, Crashes: 3, Epsilon: 0.0920, Time: 4.31s
Episode 3, Total Reward: 422.86, Crashes: 20, Epsilon: 0.0154, Time: 18.50s
Episode 4, Total Reward: 56.21, Crashes: 3, Epsilon: 0.0108, Time: 4.30s
Checkpoint saved: checkpoints_simple_right_turn\checkpoint_episode_5_step_1000.pth
Episode 5, Total Reward: 26.66, Crashes: 6, Epsilon: 0.0100, Time: 4.30s
Episode 6, Total Reward: 37.85, Crashes: 5, Epsilon: 0.0100, Time: 5.05s
Episode 7, Total Reward: 49.54, Crashes: 4, Epsilon: 0.0100, Time: 4.85s
Episode 8, Total Reward: -90.37, Crashes: 20, Epsilon: 0.0100, Time: 16.00s
Episode 9, Total Reward: 18.12, Crashes: 7, Epsilon: 0.0100, Time: 4.85s
Episode 10, Total Reward: -67.14, Crashes: 19, Epsilon: 0.0100, Time: 16.95s
Episode 11, Total Reward: 35.61, Crashes: 5, Epsilon: 0.0100, Time: 4.30s
Checkpoint saved: checkpoin

d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.31577449303586036
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Episode 40, Total Reward: -62.92, Crashes: 18, Epsilon: 0.0100, Time: 18.32s
Episode 41, Total Reward: 35.41, Crashes: 5, Epsilon: 0.0100, Time: 4.30s
Episode 42, Total Reward: 46.45, Crashes: 4, Epsilon: 0.0100, Time: 4.30s
Episode 43, Total Reward: 45.17, Crashes: 4, Epsilon: 0.0100, Time: 4.30s
Episode 44, Total Reward: 36.87, Crashes: 5, Epsilon: 0.0100, Time: 4.50s
Episode 45, Total Reward: 8.37, Crashes: 8, Epsilon: 0.0100, Time: 5.95s
Episode 46, Total Reward: 45.80, Crashes: 4, Epsilon: 0.0100, Time: 4.30s
Episode 47, Total Reward: 38.24, Crashes: 5, Epsilon: 0.0100, Time: 4.60s
Checkpoint saved: checkpoints_simple_right_turn\checkpoint_episode_48_step_7000.pth
Episode 48, Total Reward: -77.89, Crashes: 19, Epsilon: 0.0100, Time: 17.25s
Episode 49, Total Reward: 57.23, Crashes: 3, Epsilon: 0.0100, Time: 4.30s
Episode 50, Total Reward: 28.72, Crashes: 6, Epsilon: 0.0100, Time: 4.30s
Episode 51, Total Reward: 45.85, Crashes: 4, Epsilon: 0.0100, Time: 4.30s
Episode 52, Total Rewar

...

d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.30562369781546295
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Episode 3119, Total Reward: 452.87, Crashes: 17, Epsilon: 0.0100, Time: 12.11s
Episode 3120, Total Reward: 622.86, Crashes: 0, Epsilon: 0.0100, Time: 9.75s
Episode 3121, Total Reward: 522.86, Crashes: 10, Epsilon: 0.0100, Time: 12.35s
Episode 3122, Total Reward: 592.86, Crashes: 3, Epsilon: 0.0100, Time: 10.00s
Episode 3123, Total Reward: 622.86, Crashes: 0, Epsilon: 0.0100, Time: 9.80s
Checkpoint saved: checkpoints_simple_right_turn\checkpoint_episode_3124_step_679000.pth
Episode 3124, Total Reward: 622.86, Crashes: 0, Epsilon: 0.0100, Time: 10.70s
Episode 3125, Total Reward: 622.86, Crashes: 0, Epsilon: 0.0100, Time: 10.05s
Episode 3126, Total Reward: 622.86, Crashes: 0, Epsilon: 0.0100, Time: 9.90s
Episode 3127, Total Reward: 522.86, Crashes: 10, Epsilon: 0.0100, Time: 10.50s
Episode 3128, Total Reward: 622.87, Crashes: 0, Epsilon: 0.0100, Time: 9.75s
Checkpoint saved: checkpoints_simple_right_turn\checkpoint_episode_3129_step_680000.pth
Episode 3129, Total Reward: 582.86, Crashes: 

d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.27821669809054583
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.21680429961998016
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.40855319995898753
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 1.7456359999487177
  warnings.warn(f"Time-step timed out. Elapsed since last time-st

Episode 3131, Total Reward: 80.49, Crashes: 1, Epsilon: 0.0100, Time: 6.95s


d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6332298995694146
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 2.071740399929695
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.4892456999514252
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
d:\mini_conda\envs\collabkart\Lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.20661210000980645
  warnings.warn(f"Time-step timed out. Elapsed since last time-step:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import deque
import os
from tmrl import get_environment
import time
# Assume device is defined earlier in the code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ComplexDQN(nn.Module):
    def __init__(self, h, w, outputs):
        super(ComplexDQN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        
        # Calculate the size of flattened features
        conv_out_size = self._get_conv_out(h, w)
        
        # Fully connected layers
        self.fc1 = nn.Linear(conv_out_size + 9, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, outputs)
        
        # Dropout layers
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.3)
        
        # Attention mechanism
        self.attention = nn.MultiheadAttention(embed_dim=128, num_heads=4)

    def _get_conv_out(self, h, w):
        o = F.relu(self.bn1(self.conv1(torch.zeros(1, 4, h, w))))
        o = F.relu(self.bn2(self.conv2(o)))
        o = F.relu(self.bn3(self.conv3(o)))
        o = F.relu(self.bn4(self.conv4(o)))
        return int(np.prod(o.size()))

    def forward(self, x, additional_inputs):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        
        # Apply attention mechanism
        x = x.view(x.size(0), x.size(1), -1)  # Reshape for attention
        x, _ = self.attention(x, x, x)
        x = x.view(x.size(0), -1)  # Flatten
        
        # Concatenate with additional inputs
        x = torch.cat((x, additional_inputs), dim=1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        return self.fc3(x)

class ComplexDQNAgent:
    def __init__(self, n_actions, memory_size=100000, batch_size=64, gamma=0.99, 
                 epsilon_start=1.0, epsilon_final=0.01, epsilon_decay=0.995, 
                 learning_rate=0.0005, target_update=10, 
                 checkpoint_dir="checkpoints_complex"):
        self.n_actions = n_actions
        self.memory = deque(maxlen=memory_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_final = epsilon_final
        self.epsilon_decay = epsilon_decay
        self.target_update = target_update
        self.checkpoint_dir = checkpoint_dir
        self.steps = 0

        # Initialize networks
        self.policy_net = ComplexDQN(64, 64, n_actions).to(device)
        self.target_net = ComplexDQN(64, 64, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        # Initialize optimizer and learning rate scheduler
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10000, gamma=0.95)

    def select_action(self, state):
        if random.random() > self.epsilon:
            with torch.no_grad():
                image = torch.FloatTensor(state[3]).unsqueeze(0).to(device)
                additional = torch.FloatTensor(np.concatenate([state[0], state[1], state[2], state[4], state[5]])).unsqueeze(0).to(device)
                return self.policy_net(image, additional).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = zip(*batch)

        state_image_batch = torch.FloatTensor(np.stack([s[3] for s in state_batch])).to(device)
        state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in state_batch])).to(device)

        next_state_image_batch = torch.FloatTensor(np.stack([s[3] for s in next_state_batch])).to(device)
        next_state_additional_batch = torch.FloatTensor(np.stack([np.concatenate([s[0], s[1], s[2], s[4], s[5]]) for s in next_state_batch])).to(device)

        action_batch = torch.LongTensor(action_batch).to(device)
        reward_batch = torch.FloatTensor(reward_batch).to(device)
        done_batch = torch.FloatTensor(done_batch).to(device)

        # Compute Q(s_t, a) - the model computes Q(s_t), then we select the columns of actions taken
        state_action_values = self.policy_net(state_image_batch, state_additional_batch).gather(1, action_batch.unsqueeze(1))

        # Compute V(s_{t+1}) for all next states
        next_state_values = self.target_net(next_state_image_batch, next_state_additional_batch).max(1)[0].detach()

        # Compute the expected Q values
        expected_state_action_values = reward_batch + (1 - done_batch) * self.gamma * next_state_values

        # Compute Huber loss
        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), max_norm=1.0)
        self.optimizer.step()
        self.scheduler.step()

        # Update epsilon
        self.epsilon = max(self.epsilon_final, self.epsilon * self.epsilon_decay)

    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    def save_checkpoint(self, episode):
        if not os.path.exists(self.checkpoint_dir):
            os.makedirs(self.checkpoint_dir)
        checkpoint = {
            'episode': episode,
            'model_state_dict': self.policy_net.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'epsilon': self.epsilon,
            'steps': self.steps
        }
        filename = os.path.join(self.checkpoint_dir, f"checkpoint_episode_{episode}_step_{self.steps}.pth")
        torch.save(checkpoint, filename)
        print(f"Checkpoint saved: {filename}")

    def load_checkpoint(self, filename):
        checkpoint = torch.load(filename)
        self.policy_net.load_state_dict(checkpoint['model_state_dict'])
        self.target_net.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.epsilon = checkpoint['epsilon']
        self.steps = checkpoint['steps']
        return checkpoint['episode']

    def train(self, env, num_episodes, max_steps_per_episode=1000):
        for episode in range(num_episodes):
            state = env.reset()
            total_reward = 0
            
            for step in range(max_steps_per_episode):
                action = self.select_action(state)
                next_state, reward, done, _ = env.step(action.item())
                
                self.remember(state, action.item(), reward, next_state, done)
                self.replay()
                
                state = next_state
                total_reward += reward
                self.steps += 1
                
                if done:
                    break
            
            if episode % self.target_update == 0:
                self.update_target_network()
                self.save_checkpoint(episode)
            
            print(f"Episode {episode}, Total Reward: {total_reward}, Epsilon: {self.epsilon:.2f}")

    def evaluate(self, env, num_episodes):
        self.policy_net.eval()
        total_rewards = []

        for _ in range(num_episodes):
            state = env.reset()
            episode_reward = 0
            done = False

            while not done:
                with torch.no_grad():
                    image = torch.FloatTensor(state[3]).unsqueeze(0).to(device)
                    additional = torch.FloatTensor(np.concatenate([state[0], state[1], state[2], state[4], state[5]])).unsqueeze(0).to(device)
                    action = self.policy_net(image, additional).max(1)[1].view(1, 1)
                next_state, reward, done, _ = env.step(action.item())
                episode_reward += reward
                state = next_state

            total_rewards.append(episode_reward)

        self.policy_net.train()
        return np.mean(total_rewards), np.std(total_rewards)

def preprocess_observation(obs):
    speed = np.array(obs[0]).flatten()
    steering = np.array(obs[1]).flatten()
    gyro = np.array(obs[2]).flatten()
    image = np.array(obs[3]).astype(np.float32) / 255.0
    prev_action = np.array(obs[4]).flatten()
    action = np.array(obs[5]).flatten()
    
    speed = np.clip(speed / 300.0, -1, 1)
    steering = np.clip(steering / np.pi, -1, 1)
    gyro = np.clip(gyro / np.pi, -1, 1)
    prev_action = np.clip(prev_action, -1, 1)
    action = np.clip(action, -1, 1)
    
    return (speed, steering, gyro, image, prev_action, action)

def env_action_to_agent_action(env_action, n_actions):
    if isinstance(env_action, (int, np.integer)):
        return env_action
    elif isinstance(env_action, (float, np.float32, np.float64)):
        return int((env_action + 1) * (n_actions - 1) / 2)
    elif isinstance(env_action, np.ndarray):
        return np.argmax(env_action)
    else:
        raise ValueError(f"Unexpected action type: {type(env_action)}")

def agent_action_to_env_action(agent_action, n_actions):
    steering = (agent_action * 2 / (n_actions - 1)) - 1
    throttle = 1.0
    brake = 0.0
    return [throttle, brake, steering]

def detect_crash(obs, prev_obs, speed_threshold=1.0):
    if prev_obs is None:
        return False
    
    current_speed = np.linalg.norm(obs[0])
    prev_speed = np.linalg.norm(prev_obs[0])
    
    if prev_speed - current_speed > speed_threshold:
        return True
    
    return False

def train(env, agent, num_episodes, max_steps_per_episode):
    for episode in range(num_episodes):
        obs, _ = env.reset()
        state = preprocess_observation(obs)
        total_reward = 0
        episode_start_time = time.time()
        prev_obs = None
        crashes = 0

        for step in range(max_steps_per_episode):
            agent_action = agent.select_action(state)
            env_action = agent_action_to_env_action(agent_action.item(), agent.n_actions)
            next_obs, env_reward, terminated, truncated, _ = env.step(env_action)
            next_state = preprocess_observation(next_obs)
            
            if detect_crash(next_obs, prev_obs):
                crashes += 1
                env_reward-= 10
            done = terminated or truncated
            
            elapsed_time = time.time() - episode_start_time
            time_factor = 1 / (elapsed_time + 1)
            reward = (env_reward)
            
            agent.remember(state, agent_action.item(), reward, next_state, done)
            agent.replay()
            
            state = next_state
            prev_obs = next_obs
            total_reward += reward

            agent.steps += 1

            if agent.steps % 1000 == 0:
                agent.save_checkpoint(episode)

            if done:
                break
        # total_reward-= 20*crashes
        if episode % 10 == 0:
            agent.update_target_network()

        print(f"Episode {episode}, Total Reward: {total_reward:.2f}, Crashes: {crashes}, Epsilon: {agent.epsilon:.4f}, Time: {elapsed_time:.2f}s")


In [2]:
if __name__ == "__main__":
    env = get_environment()
    n_actions = 3
    agent = ComplexDQNAgent(n_actions)
    agent.load_checkpoint("checkpoints_complex\checkpoint_episode_365_step_73000.pth")
    train(env, agent, num_episodes=5000, max_steps_per_episode=5000)

d:\mini_conda\envs\collabkart\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Error(s) in loading state_dict for ComplexDQN:
	Missing key(s) in state_dict: "conv4.weight", "conv4.bias", "bn4.weight", "bn4.bias", "bn4.running_mean", "bn4.running_var", "attention.in_proj_weight", "attention.in_proj_bias", "attention.out_proj.weight", "attention.out_proj.bias". 
	size mismatch for fc1.weight: copying a param with shape torch.Size([512, 1033]) from checkpoint, the shape in current model is torch.Size([512, 2057]).